In [51]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pickle
from mutation import *
from crossover import *
from rmp import *
from mfea import *
from task import *
from llm import *
import matplotlib.pyplot as plt
from dotenv import load_dotenv
import os
import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
for task_id in range(5, 9):
	cec2017_tasks, task_name = get_2_tasks(task_id)
	num_run = 10
	for i in range(num_run):
		np.random.seed(i)
		num_indis_per_task = 100
		indi_len = 50
		mutation = PolynomialMutation(5, 1.0 / indi_len)
		crossover = SBXCrossover(mutation, eta=2)
		rmp = AdaptiveRMPMatrix(rmp_pop_size=5, num_gen=5, pc=0.8, pm=0.1)
		# rmp = NormalRMP(0.3)
		amfea = AMFEA(num_indis_per_task, indi_len, cec2017_tasks, crossover, mutation, rmp)

		bfs, mfs = amfea.fit(num_gen=1000, monitor=True, monitor_rate=100, llm_rate=100)

		df = pd.DataFrame(bfs.T)
		df.to_csv("result/" + "llm/" + task_name + "/" + task_name + "{0}.csv".format(i))

CI_H
Initialization:
Task 0:
Best Fitness: 33.260853059944985
Mean Fitness: 43.191517893469815

Task 1:
Best Fitness: 28389.17824247567
Mean Fitness: 42183.975985972975

Creating strategy
Creating strategy
Creating strategy
Creating strategy
Creating strategy
Evaluating strategy
Strategy: 
Calculate the Mahalanobis distance between the mean vectors of each task to quantify the dissimilarity in distribution. Adjust RMP values inversely proportional to the distance, assigning higher probabilities to tasks with more similar distributions.
Analyze the covariance matrices of the population for each task to identify tasks with similar spread or diversity. Increase RMP values between tasks with similar variances to encourage mating between individuals in those tasks.
Cluster the tasks based on the population mean vectors using a hierarchical clustering algorithm. Adjust RMP values based on the cluster assignments, promoting mating between tasks within the same cluster to exploit potential sim

KeyboardInterrupt: 